# Declare dependancies of the project

In [2]:
:dep itertools
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
:dep splines
:dep csv
use itertools::Itertools;
use splines::Key;
use std::{
    collections::HashMap,
    fs::File,
    hash::BuildHasher,
    io::{self, BufReader},
};
extern crate plotters;
use plotters::prelude::*;

# Data importing
All data comes from the csv files in ./data and goes from 1990 to 2021

In [3]:
const START_YEAR: usize = 1990;
const END_YEAR: usize = 2021;

Samples a data point using the given year.

In [4]:
fn value_from_year(year: usize, data: &[f64]) -> Option<f64> {
    year.checked_sub(START_YEAR)
        .and_then(|index| data.get(index).copied())
}

Load HDI data from ./data/HDI.csv [Source](https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv)

In [23]:
fn load_hdi() -> io::Result<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/HDI.csv")?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Ok(csv
        .records()
        .flatten()
        .filter_map(|row| {
            row.get(1).and_then(|name| {
                (5..37)
                    .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                    .collect::<Option<Vec<_>>>()
                    .map(|data| (name.to_string(), data))
            })
        })
        .collect::<HashMap<String, Vec<f64>>>())
}

Load GDP per capita, ppp adjusted, data from ./data/GDP.csv [Source](https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.PP.CD?downloadformat=csv)

In [22]:
fn load_gdp() -> io::Result<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/GDP.csv")?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Ok(csv
        .records()
        .flatten()
        .filter_map(|row| {
            row.get(0).and_then(|name| {
                (1..33)
                    .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                    .collect::<Option<Vec<_>>>()
                    .map(|data| (name.to_string(), data))
            })
        })
        .collect::<HashMap<String, Vec<f64>>>())
}

## Filling in the gaps
The gini coeficint data is sparse as it is a relativly difficult indicator to measure and many countries have only a few or no data points what so ever. It seems tempting to omit countries with little data, but since this investigation is looking at development it would spoil the results to do so, as the countries with little data tend to be the ones low in thier development. The Gini coeficeint dosn't change much year on year so I think it is reasonable to interploate the availible data to do our investigation. To do this we will construct splines and sample them.

In [6]:
fn sample_spline(all: Vec<(usize, Option<f64>)>, spline: splines::Spline<f64, f64>) -> Vec<f64> {
    all.iter()
        .map(|(year, value)| {
            value.map_or_else(
                || spline.clamped_sample((*year + START_YEAR) as f64).unwrap(),
                |value| value,
            )
        })
        .collect_vec()
}

This function loads the data, and fills in the gaps with splines

In [7]:
fn load_gini() -> Option<HashMap<String, Vec<f64>>> {
    let file = File::open("./data/Gini.csv").ok()?;
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    Some(
        csv.records()
            .flatten()
            .filter_map(|row| {
                row.get(0).and_then(|name| {
                    let all = (1..33)
                        .map(|i| row.get(i).and_then(|s| s.parse::<f64>().ok()))
                        .enumerate()
                        .collect_vec();

                    let keys = all
                        .iter()
                        .filter_map(|(y, v)| v.map(|value| (value, (y + START_YEAR) as f64)))
                        .map(|(v, y)| Key::new(y, v, splines::Interpolation::Cosine))
                        .collect_vec();

                    if keys.is_empty() {
                        None
                    } else {
                        let spline = splines::Spline::from_vec(keys);

                        Some((name.to_string(), sample_spline(all, spline)))
                    }
                })
            })
            .collect::<HashMap<String, Vec<f64>>>(),
    )
}

## Example interpolation
Here we have the Gini data for Bangladesh. 
- The blue points are the acutal data
- The green are the interpolated points

In [21]:
let mut data: Vec<(f64,f64)> = vec![];
{
    let gini_data = load_gini().unwrap();
    let bangladesh = gini_data.get("Bangladesh").expect("Bangladesh not in data set");
    for year in START_YEAR..END_YEAR {
        data.push(((year ) as f64, value_from_year(year, bangladesh).unwrap()));
    }
}
let mut raw_data: Vec<(f64,f64)> = vec![];
{
    let file = File::open("./data/Gini.csv").ok().unwrap();
    let buffer = BufReader::new(file);
    let mut csv = csv::Reader::from_reader(buffer);
    for record in csv.records().flatten() {
        if record.get(0).is_some_and(|name| name != "Bangladesh") {
            continue;
        }         
        raw_data = (1..33).map(|i| record.get(i).and_then(|s| s.parse::<f64>().ok())).enumerate().filter_map(|(i,v)|{
            let year = (i + START_YEAR) as f64;
            v.map(|v| (year,v))
        }).collect_vec();
    }
};



evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Gini for bangladesh", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged((START_YEAR as f64)..(END_YEAR as f64), 20.0..40.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .y_desc("%Gini")
        .x_desc("Year")
        .draw()?;
    
    chart.draw_series(data.iter().map(|(x,y)| Circle::new((*x,*y), 3, GREEN.filled())));
    chart.draw_series(raw_data.iter().map(|(x,y)| Circle::new((*x,*y), 3, BLUE.filled())));

    
    Ok(())
})

Gini for bangladesh
 
 
%Gini
 
 
Year
 
 
 
20.0
 
 
 
22.0
 
 
 
24.0
 
 
 
26.0
 
 
 
28.0
 
 
 
30.0
 
 
 
32.0
 
 
 
34.0
 
 
 
36.0
 
 
 
38.0
 
 
 
40.0
 
 
 
 
1990.0
 
 
 
1995.0
 
 
 
2000.0
 
 
 
2005.0
 
 
 
2010.0
 
 
 
2015.0
 
 
 
2020.0

# Using the data
First well load  the data using the functions defined before

In [24]:
let hdi_data = load_hdi().unwrap();
let gdp_data = load_gdp().unwrap();
let gini_data = load_gini().unwrap();

## Plotting
First lets plot the HDI against the Gini

In [25]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("HDI vs Gnii", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0f64..100f64, 0.2..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .x_desc("%Gini")
        .y_desc("HDI")
        .draw()?;
    

    let mut data: Vec<(f64,f64,f64)> = vec![];
    let mut i = 0.0;
    for (name, hdi_points) in hdi_data.iter().skip(50) {
        if let Some(gini_points) = gini_data.get(name) {
            for (x,y) in gini_points.iter().zip(hdi_points.iter()) {
                data.push((*x,*y,i));
            }
        }
        i = i + 0.1 ;
    }

    let _ = chart.draw_series(data.iter().map(|(x,y,c)| Circle::new((*x,*y), 3, HSLColor(240.0 / 360.0 - 240.0 / 360.0 * c / 5.0,1.0,0.7).filled())));

    
    Ok(())
})

HDI vs Gnii
 
 
HDI
 
 
%Gini
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0

Not exactly crystal clear, lets plot it in 3d, letting us see how it evolves...

In [26]:
evcxr_figure((1000, 780), |root| {
    let root = root.titled("HDI vs Gini coeficent over time.", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(20.0..70.0, 0.3..0.9, 1990.0..2021.0)?;
    
        chart.with_projection(|mut p| {
            p.pitch = 0.6; 
            p.yaw = 0.4;
            p.scale = 0.7;
            p.into_matrix() // build the projection matrix
        });

    
    chart.configure_axes().draw()?;

    let mut c = 0.0;
    for (name, hdi_points) in hdi_data.iter() {
        if let Some(gini_points) = gini_data.get(name) {
            let mut line = vec![];
            for (i,(x,y)) in gini_points.iter().zip(hdi_points.iter()).enumerate() {
                line.push((*x,*y,i as f64 + START_YEAR as f64));
            }
                chart.draw_series(LineSeries::new(
        line.iter().map(|point| *point),
        &HSLColor(240.0 / 360.0 - 240.0 / 360.0 * c / 5.0,1.0,0.7),
    ))?;
        c = ( c + 0.1 ) % 5.0 ;
        }
    }  
    
    Ok(())
})

HDI vs Gini coeficent over time.
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
1990.0
 
 
 
1995.0
 
 
 
2000.0
 
 
 
2005.0
 
 
 
2010.0
 
 
 
2015.0
 
 
 
2020.0

The from the data we can see that there is no clear relationship between HDI and Gini numbers, and is therfore likely to be little corelation between development and income equaltiy.
# Part 2
So the kuznets was wrong right? Well the theory is about equality, but not nessisarily wealth or income equality.
To better understand what i'm talking about, lets look at HDI vs GDP data.

In [19]:
let mut data: Vec<(f64,f64)> = vec![];
for (name, hdi_points) in hdi_data.iter() {
    if let Some(gdp_points) = gdp_data.get(name) {
        for (x,y) in gdp_points.iter().zip(hdi_points.iter()) {
            data.push((*x,*y));
        }
    }
}

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("HDI vs Real GDP PPP PC", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0f64..10e4f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    // Draw data points
    let _ = chart.draw_series(data.iter().map(|(x,y)| Circle::new((*x,*y), 3, RGBAColor(255,100,120,0.2).filled())));
    // Draw 1 / x
    let _ = chart.draw_series(LineSeries::new(
        (0..700).map(|x| {
            let x = x as f64 ;
            let y = 1. / -(x*0.05+ 1.2) + 1.0;
            (x * 200.0 ,y * 1.)
            } ),  BLUE
    ));
    
    Ok(())
})

Error: cannot find value `hdi_data` in this scope

Error: cannot find value `gdp_data` in this scope

Error: use of deprecated method `plotters::chart::ChartBuilder::<'a, 'b, DB>::build_ranged`: `build_ranged` has been renamed to `build_cartesian_2d` and is to be removed in the future.

Here we can clearly see the inverse falloff relationship between development and GDP. This is caused by the same mechanism that causes marginal utiliy to fall. As countries become more developed, the additional reasouces reqired to develop further increase exponentialy. And while this is a macro example, the same principle applies at the micro level within individual countries. So despite the fact absolute wealth dispartiy may increase with develpment, it's posible that the overal equality may trend in similar fasion to the kuznetz cuvre. 

# Moddling equality

## Gini and the lorenz curve
Earlyer in the year I devised a simple expresion for plotting lorenz curves $ x^{k} $
- Where k is a constant and $ k = \frac{-2}{ G -1 } -1 $
- G is the  Gini coeficeint
This curve fufils the property $ G = \frac{A}{A+B} $ or more simply $ G = 1 - 2B $  
Example: USA with G = 0.48

In [13]:
let g = 0.48;
let k = -2.0 / (g - 1.0) - 1.0 ;

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Lorenz curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..1.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    let _ = chart.draw_series(LineSeries::new(
        (0..=1000).map(|x| {
            let x = x as f64 * 0.001;
            let y = f64::powf(x,k);
            (x  ,y )
            } ),  BLUE
    ));
    let _ = chart.draw_series(LineSeries::new(
        (0..2).map(|x| {
            let x = x as f64;
            let y = x;
            (x  ,y )
            } ),  RED
    ));
    
    Ok(())
})

Lorenz curve for USA
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,439 40,439 41,439 41,439 42,439 42,439 43,439 44,439 44,439 45,439 45,439 46,439 47,439 47,439 48,439 48,439 49,439 50,439 50,439 51,439 51,439 52,439 53,439 53,439 54,439 54,439 55,439 56,439 56,439 57,439 57,439 58,439 59,439 59,439 60,439 60,439 61,439 62,439 62,439 63,439 63,439 64,439 65,439 65,439 66,439 66,439 67,439 68,439 68,439 69,439 69,439 70,439 71,439 71,439 72,439 72,439 73,439 74,439 74,439 75,439 75,439 76,439 77,439 77,439 78,439 78,439 79,439 80,439 80,439 81,439 81,439 82,439 83,439 83,439 84,439 84,439 85,439 86,439 86,439 87,439 87,439 88,439 89,439 89,439 90,439 90,439 91,439 92,439 92,439 93,439 93,439 94,439 95,439 95,439 96,439 96,439 97,439 98,439 98,439 99,439 99,439 100,439 101,439 101,439 102,439 102,439 103,439 104,439 104,439 105,439 105,439 106,439 107,439 107,439 108,439 108,439 109,439 110,439 110,439 111,439 111,439 112,438 113,438 113,438 114,438 114,438 115,438 116,438 116,438 117,438 117,438 118,438 119,438 119,438 120,438 120,438 121,438 122,438 122,438 123,438 123,438 124,438 125,438 125,438 126,438 126,438 127,438 128,438 128,438 129,438 129,438 130,438 131,438 131,438 132,437 132,437 133,437 134,437 134,437 135,437 135,437 136,437 137,437 137,437 138,437 138,437 139,437 140,437 140,437 141,437 141,437 142,437 143,437 143,437 144,437 144,437 145,437 146,437 146,436 147,436 147,436 148,436 149,436 149,436 150,436 150,436 151,436 152,436 152,436 153,436 153,436 154,436 155,436 155,436 156,436 156,436 157,436 158,435 158,435 159,435 159,435 160,435 160,435 161,435 162,435 162,435 163,435 163,435 164,435 165,435 165,435 166,435 166,435 167,434 168,434 168,434 169,434 169,434 170,434 171,434 171,434 172,434 172,434 173,434 174,434 174,434 175,434 175,433 176,433 177,433 177,433 178,433 178,433 179,433 180,433 180,433 181,433 181,433 182,433 183,432 183,432 184,432 184,432 185,432 186,432 186,432 187,432 187,432 188,432 189,432 189,432 190,431 190,431 191,431 192,431 192,431 193,431 193,431 194,431 195,431 195,431 196,430 196,430 197,430 198,430 198,430 199,430 199,430 200,430 201,430 201,430 202,429 202,429 203,429 204,429 204,429 205,429 205,429 206,429 207,429 207,428 208,428 208,428 209,428 210,428 210,428 211,428 211,428 212,428 213,427 213,427 214,427 214,427 215,427 216,427 216,427 217,427 217,426 218,426 219,426 219,426 220,426 220,426 221,426 222,426 222,425 223,425 223,425 224,425 225,425 225,425 226,425 226,424 227,424 228,424 228,424 229,424 229,424 230,424 231,424 231,423 232,423 232,423 233,423 234,423 234,423 235,422 235,422 236,422 237,422 237,422 238,422 238,422 239,421 240,421 240,421 241,421 241,421 242,421 243,420 243,420 244,420 244,420 245,420 246,420 246,420 247,419 247,419 248,419 249,419 249,419 250,419 250,418 251,418 252,418 252,418 253,418 253,417 254,417 255,417 255,417 256,417 256,417 257,416 258,416 258,416 259,416 259,416 260,415 261,415 261,415 262,415 262,415 263,415 264,414 264,414 265,414 265,414 266,414 267,413 267,413 268,413 268,413 269,413 270,412 270,412 271,412 271,412 272,412 273,411 273,411 274,411 274,411 275,411 276,410 276,410 277,410 277,410 278,409 279,409 279,409 280,409 280,409 281,408 281,408 282,408 283,408 283,408 284,407 284,407 285,407 286,407 286,406 287,406 287,406 288,406 289,405 289,405 290,405 290,405 291,405 292,404 292,404 293,404 293,404 294,403 295,403 295,403 296,403 296,402 297,402 298,402 298,402 299,401 299,401 300,401 301,401 301,400 302,400 302,400 303,400 304,399 304,399 305,399 305,399 306,398 307,398 307,398 308,397 308,397 309,397 310,397 310,396 311,396 311,396 312,396 313,395 313,395 314,395 314,394 315,394 316,394 316,394 317,393 317,393 318,393 319,392 319,392 320,392 320,392 321,391 322,391 32

## Creating an income distribution from the lorenz curve
The lorenz curve shows cumlative values, or in other words, the intergral of the individual incomes. Therfore to reconstruct a modle of the incomes all we need to do is take the deriviative of the curve and multiply by the total amount of real wealth to get at the values. Ill be using real GDP PPP adjusted per capita to provide a stand in for this figure. The math is as follows
$ I(x) = \frac{d}{dx} x^{k} = kx^{k-1} $
Conviently the area under this curve from 0 to 1 is always 1. So to get real wealth... $ R(x) = gkx^{k-1} $ 
- Where g is GDP PPP PC
Example plot of the USA:

In [14]:
let gdp: f64 = gdp_data.get("United States").unwrap()[31];

evcxr_figure((700, 700), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Real wealth curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..20e4)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    let _ = chart.draw_series(LineSeries::new(
        (0..1000).map(|x| {
            let x = x as f64 * 0.001;
            let y = gdp*k*x.powf( k - 1.0 );
            (x  ,y )
            } ),  GREEN
    ));
    
    Ok(())
})

Error: cannot find value `gdp_data` in this scope

Error: use of deprecated method `plotters::chart::ChartBuilder::<'a, 'b, DB>::build_ranged`: `build_ranged` has been renamed to `build_cartesian_2d` and is to be removed in the future.

## Moddling utility

From the law of decreasing marginal utility we know
$ \Delta U = e^{kc} $ where 
- U is utility
- c is conumption or any other source of utility
- k is some negative constant

And therfore U as the intergral of marginal uitilty is $ U = \int_{0}^{c} e^{kc} \, dc =  \frac{e^{kc}-1}{k} $


## Putting it together
Now we have the quantity of real wealth, and model of how that translates to utility. We'll now combine them by inserting the former into the latter  
### $\frac{ e^{ Ugkx^{ k - 1 } } - 1 }{ U }$
Example plot where $ U = -0.0001 $

In [17]:
let u = -0.0001;

evcxr_figure((700, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Utility wealth curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..10000.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    let _ = chart.draw_series(LineSeries::new(
        (0..1000).map(|x| {
            let x = x as f64 * 0.001;
            let y = ( ( gdp * u * k * x.powf(k - 1.0) ).exp() - 1.0 ) / u ;
            (x  ,y )
            } ),  GREEN
    ));
    
    Ok(())
})

Utility wealth curve for USA
 
 
 
0.0
 
 
 
1000.0
 
 
 
2000.0
 
 
 
3000.0
 
 
 
4000.0
 
 
 
5000.0
 
 
 
6000.0
 
 
 
7000.0
 
 
 
8000.0
 
 
 
9000.0
 
 
 
10000.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="40,259 40,259 41,259 41,259 42,259 43,259 43,259 44,259 45,259 45,259 46,259 47,258 47,258 48,258 49,258 49,258 50,257 51,257 51,257 52,256 53,256 53,256 54,255 55,255 55,255 56,254 57,254 57,254 58,253 59,253 59,252 60,252 61,252 61,251 62,251 63,250 63,250 64,249 65,249 65,248 66,248 67,247 67,246 68,246 68,245 69,245 70,244 70,244 71,243 72,242 72,242 73,241 74,240 74,240 75,239 76,238 76,238 77,237 78,236 78,236 79,235 80,234 80,234 81,233 82,232 82,231 83,231 84,230 84,229 85,228 86,228 86,227 87,226 88,225 88,224 89,224 90,223 90,222 91,221 92,220 92,219 93,219 94,218 94,217 95,216 96,215 96,214 97,213 97,213 98,212 99,211 99,210 100,209 101,208 101,207 102,206 103,205 103,205 104,204 105,203 105,202 106,201 107,200 107,199 108,198 109,197 109,196 110,195 111,194 111,193 112,192 113,191 113,191 114,190 115,189 115,188 116,187 117,186 117,185 118,184 119,183 119,182 120,181 121,180 121,179 122,178 123,177 123,176 124,175 125,174 125,173 126,172 126,171 127,170 128,169 128,168 129,167 130,167 130,166 131,165 132,164 132,163 133,162 134,161 134,160 135,159 136,158 136,157 137,156 138,155 138,154 139,153 140,152 140,151 141,150 142,149 142,148 143,148 144,147 144,146 145,145 146,144 146,143 147,142 148,141 148,140 149,139 150,138 150,137 151,137 152,136 152,135 153,134 154,133 154,132 155,131 155,130 156,129 157,129 157,128 158,127 159,126 159,125 160,124 161,123 161,122 162,122 163,121 163,120 164,119 165,118 165,117 166,117 167,116 167,115 168,114 169,113 169,113 170,112 171,111 171,110 172,109 173,109 173,108 174,107 175,106 175,106 176,105 177,104 177,103 178,102 179,102 179,101 180,100 181,100 181,99 182,98 183,97 183,97 184,96 184,95 185,95 186,94 186,93 187,92 188,92 188,91 189,90 190,90 190,89 191,88 192,88 192,87 193,86 194,86 194,85 195,85 196,84 196,83 197,83 198,82 198,81 199,81 200,80 200,80 201,79 202,78 202,78 203,77 204,77 204,76 205,76 206,75 206,74 207,74 208,73 208,73 209,72 210,72 210,71 211,71 212,70 212,70 213,69 213,69 214,68 215,68 215,67 216,67 217,66 217,66 218,65 219,65 219,64 220,64 221,63 221,63 222,62 223,62 223,62 224,61 225,61 225,60 226,60 227,59 227,59 228,59 229,58 229,58 230,57 231,57 231,57 232,56 233,56 233,55 234,55 235,55 235,54 236,54 237,54 237,53 238,53 239,53 239,52 240,52 240,52 241,51 242,51 242,51 243,50 244,50 244,50 245,49 246,49 246,49 247,48 248,48 248,48 249,47 250,47 250,47 251,47 252,46 252,46 253,46 254,45 254,45 255,45 256,45 256,44 257,44 258,44 258,44 259,43 260,43 260,43 261,43 262,42 262,42 263,42 264,42 264,42 265,41 266,41 266,41 267,41 268,40 268,40 269,40 269,40 270,40 271,39 271,39 272,39 273,39 273,39 274,38 275,38 275,38 276,38 277,38 277,38 278,37 279,37 279,37 280,37 281,37 281,37 282,36 283,36 283,36 284,36 285,36 285,36 286,36 287,35 287,35 288,35 289,35 289,35 290,35 291,35 291,34 292,34 293,34 293,34 294,34 295,34 295,34 296,34 297,33 297,33 298,33 298,33 299,33 300,33 300,33 301,33 302,33 302,32 303,32 304,32 304,32 305,32 306,32 306,32 307,32 308,32 308,32 309,32 310,31 310,31 311,31 312,31 312,31 313,31 314,31 314,31 315,31 316,31 316,31 317,31 318,31 318,30 319,30 320,30 320,30 321,30 322,30 322,30 323,30 324,30 324,30 325,30 326,30 326,30 327,30 327,30 328,29 329,29 329,29 330,29 331,29 331,29 332,29 333,29 333,29 334,29 335,29 335,29 336,29 337,29 337,29 338,29 339,29 339,29 340,29 341,29 341,29 342,29 343,28 343,28 344,28 345,28 345,28 346,28 347,28 347,28 348,28 349,28 349,28 350,28 351,28 351,28 352,28 353,28 353,28 354,28 355,28 355,28 356,28 356,28 357,28 358,28 358,28 359,28 360,28 360,28 361,28 362,28 362,28 363,28 364,28 364,28 365,28 366,27 366,27 367,27 368,27 368,27 369,27 370,27 3

As you can see, the power function of income combined with the fall of utility produce a sigmoid like curve.
## Measuing the inequality of the curve
In order to measure the equality of this curve we just need to reintergrate it back into a lorenz curve...  
# $ \frac{ \int_{0}^{x} e^{Ugkx^{k-1}} \,dx}{ \int_{0}^{1} e^{Ugkx^{k-1}} \,dx}$  
I was suprised to find that intergrals like these don't have anilytic solutions. Luckly we live in the 21st century and have no need to do such archic things like doing maths by hand. The computer will easly be able to solve them numericaly.

In [96]:
let u = -0.0001;

evcxr_figure((700, 700), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Utility lorenz curve for USA", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0.0..1.0, 0.0..1.0)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    let mut total = 0.0;
    let slices = 1000.0;
    let dx = 1.0 / slices; 
    
    let mut data = Vec::with_capacity(1000);
    let mut x: f64 = 0.0;
    while x < 1.0 {
        total += ( ( gdp * u * k * x.powf(k - 1.0) ).exp() - 1.0 ) / u  * dx;
        data.push((x,total));
        x += dx;
    }
    
    println!("Area under curve {:?}", data.iter().map(|v| v.1 / total * dx).sum::<f64>());
    let _ = chart.draw_series(LineSeries::new(
        data.iter().map(|(x,y)| {
            let y = y / total;
            (*x  ,y )
            } ),  GREEN
    )).unwrap().label("Utility");
    
    let _ = chart.draw_series(LineSeries::new(
        (0..=1000).map(|x| {
            let x = x as f64 * 0.001;
            let y = f64::powf(x,k);
            (x  ,y )
            } ),  BLUE
    )).unwrap().label("Income");
    
     let _ = chart.draw_series(LineSeries::new(
        (0..2).map(|x| {
            let x = x as f64;
            let y = x;
            (x  ,y )
            } ),  RED
    )).unwrap().label("Line of equality").legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &RED));
    
    Ok(())
})

Area under curve 0.4184477783530253


Utility lorenz curve for USA
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="40,659 40,659 41,659 41,659 42,659 43,659 43,659 44,659 45,659 45,659 46,659 47,659 47,659 48,659 49,659 49,659 50,659 51,659 51,659 52,659 53,659 53,659 54,659 55,659 55,659 56,659 57,659 57,659 58,659 59,659 59,659 60,659 61,659 61,659 62,659 63,659 63,659 64,659 65,659 65,659 66,659 67,659 67,659 68,659 68,659 69,659 70,659 70,659 71,659 72,658 72,658 73,658 74,658 74,658 75,658 76,658 76,658 77,658 78,658 78,658 79,658 80,658 80,658 81,657 82,657 82,657 83,657 84,657 84,657 85,657 86,657 86,657 87,657 88,656 88,656 89,656 90,656 90,656 91,656 92,656 92,656 93,655 94,655 94,655 95,655 96,655 96,655 97,655 97,654 98,654 99,654 99,654 100,654 101,654 101,653 102,653 103,653 103,653 104,653 105,653 105,652 106,652 107,652 107,652 108,652 109,651 109,651 110,651 111,651 111,650 112,650 113,650 113,650 114,650 115,649 115,649 116,649 117,649 117,648 118,648 119,648 119,648 120,647 121,647 121,647 122,647 123,646 123,646 124,646 125,645 125,645 126,645 126,645 127,644 128,644 128,644 129,643 130,643 130,643 131,642 132,642 132,642 133,642 134,641 134,641 135,641 136,640 136,640 137,640 138,639 138,639 139,638 140,638 140,638 141,637 142,637 142,637 143,636 144,636 144,636 145,635 146,635 146,634 147,634 148,634 148,633 149,633 150,632 150,632 151,632 152,631 152,631 153,630 154,630 154,630 155,629 155,629 156,628 157,628 157,627 158,627 159,627 159,626 160,626 161,625 161,625 162,624 163,624 163,623 164,623 165,622 165,622 166,622 167,621 167,621 168,620 169,620 169,619 170,619 171,618 171,618 172,617 173,617 173,616 174,616 175,615 175,615 176,614 177,614 177,613 178,613 179,612 179,612 180,611 181,611 181,610 182,609 183,609 183,608 184,608 184,607 185,607 186,606 186,606 187,605 188,605 188,604 189,603 190,603 190,602 191,602 192,601 192,601 193,600 194,599 194,599 195,598 196,598 196,597 197,597 198,596 198,595 199,595 200,594 200,594 201,593 202,592 202,592 203,591 204,591 204,590 205,589 206,589 206,588 207,588 208,587 208,586 209,586 210,585 210,585 211,584 212,583 212,583 213,582 213,581 214,581 215,580 215,579 216,579 217,578 217,578 218,577 219,576 219,576 220,575 221,574 221,574 222,573 223,572 223,572 224,571 225,570 225,570 226,569 227,568 227,568 228,567 229,566 229,566 230,565 231,564 231,564 232,563 233,562 233,562 234,561 235,560 235,560 236,559 237,558 237,558 238,557 239,556 239,556 240,555 240,554 241,554 242,553 242,552 243,552 244,551 244,550 245,549 246,549 246,548 247,547 248,547 248,546 249,545 250,545 250,544 251,543 252,543 252,542 253,541 254,540 254,540 255,539 256,538 256,538 257,537 258,536 258,535 259,535 260,534 260,533 261,533 262,532 262,531 263,530 264,530 264,529 265,528 266,528 266,527 267,526 268,525 268,525 269,524 269,523 270,522 271,522 271,521 272,520 273,520 273,519 274,518 275,517 275,517 276,516 277,515 277,514 278,514 279,513 279,512 280,512 281,511 281,510 282,509 283,509 283,508 284,507 285,506 285,506 286,505 287,504 287,503 288,503 289,502 289,501 290,500 291,500 291,499 292,498 293,498 293,497 294,496 295,495 295,495 296,494 297,493 297,492 298,492 298,491 299,490 300,489 300,489 301,488 302,487 302,486 303,486 304,485 304,484 305,483 306,483 306,482 307,481 308,480 308,480 309,479 310,478 310,477 311,477 312,476 312,475 313,474 314,474 314,473 315,472 316,471 316,471 317,470 318,469 318,468 319,468 320,467 320,466 321,465 322,464 322,464 323,463 324,462 324,461 325,461 326,460 326,459 327,458 327,458 328,457 329,456 329,455 330,455 331,454 331,453 332,452 333,452 333,451 334,450 335,449 335,449 336,448 337,447 337,446 338,446 339,445 339,444 340,443 341,442 341,442 342,441 343,440 343,439 344,439 345,438 345,437 346,436 347,436 347,435 348,434 349,433 3

##  The utility based Gini coeficient
Now that we have are lorenz curve of utility we can use $ 1 - 2B $ to obtain a new measure of inequality from the printout above we know the area is 0.4184477783530253 and the new coeficient is $ 1 - 2 * 0.4184477783530253 $ or 0.163104443. Much more equal indeed. It is also easy enough to visualy afirm this by looking at the graph)
# Utility equality vs development
Now we have all the peices lets chart development against equality

In [30]:
{
    let mut points = vec![];
    let u = -0.0001;
    for (name, gini) in &gini_data {
         let (hdi,gdp) = if let (Some(hdi),Some(gdp)) = (hdi_data.get(name),gdp_data.get(name)) {
             (hdi,gdp)
        } else {
            continue;
        };
        for year in START_YEAR..END_YEAR {
            let g = value_from_year(year, &gini).unwrap() * 0.01;
            let h = value_from_year(year, &hdi).unwrap();
            let k = -2.0 / (g - 1.0) - 1.0 ;
            let gdp = value_from_year(year, &gdp).unwrap();
            let mut total = 0.0;
            let slices = 1000.0;
            let dx = 1.0 / slices; 
    
            let mut data = Vec::with_capacity(1000);
            let mut x: f64 = 0.0;
            while x < 1.0 {
            total += ( ( gdp * u * k * x.powf(k - 1.0) ).exp() - 1.0 ) / u  * dx;
                data.push((x,total));
                x += dx;
            }
            let area = data.iter().map(|v| v.1 ).sum::<f64>() / total * dx ;
            let c = 1.0 - 2.0 * area;
            //println!("c {c} total {total} g {g} h {h} k {k} gdp {gdp} ");
            points.push((h,c));
        }
    }
    evcxr_figure((700, 700), |root| {
        let mut chart = ChartBuilder::on(&root)
            .caption("Equality vs HDI", ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_ranged(0.0..1.0, 0.0..1.0)?;
    
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .draw()?;
        let _ = chart.draw_series(points.iter().map(|(x,y)| Circle::new((*x,*y), 3, RGBAColor(255,100,120,0.2).filled())));
        let _ = chart.draw_series(LineSeries::new(
            (0..1000).map(|x| {
                
                
            let x = x as f64* 0.001;
            let y = 0.4 - 1.8 * (x - 0.5).powf(2.0);
                (x,y)
                }
            ), GREEN
        ));
        let _ = chart.draw_series(LineSeries::new(
            (0..1000).map(|x| {
                
                
            let x = x as f64* 0.001;
            let y = 0.6 - (3.0 * x - 3.4).exp();
                (x,y)
                }
            ), BLUE
        ));
        Ok(())
    })
}

Equality vs HDI
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


# Conlusion
Unfortunaltely, the back end of the graph appears to be missing, since we don't have data o extreamly under developed countries. However this may not be an issue since most of the debate was regarding the crest and then decline in inequality after a certain point. I hope this iliustrates the idea that it could be possible to gain increased equality with development, even despite less equal material real income.